In [1]:
# !pip install scikit-surprise

  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-win_amd64.whl size=713322 sha256=9fef54a20cc2544bda749be21140f096d3c54c7ca04932fa3c1acb3f7222fe01
  Stored in directory: c:\users\oh.chi.hooooon\appdata\local\pip\cache\wheels\76\44\74\b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [2]:
from surprise import Dataset

In [7]:
# Dataset.load_builtin('ml-100k')는 터미널에서 한 번 실행하고 노트북에서 다시 실행
data = Dataset.load_builtin("ml-100k")

In [9]:
type(data)

surprise.dataset.DatasetAutoFolds

In [10]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data)

In [12]:
type(trainset), type(testset)

(surprise.trainset.Trainset, list)

In [13]:
trainset.n_users, trainset.n_items, trainset.n_ratings

(943, 1646, 80000)

In [17]:
[ x for x in trainset.all_users() ][:10]
[ x for x in trainset.all_items() ][:10]
[ x for x in trainset.all_ratings() ][:10]

[(0, 0, 5.0),
 (0, 14, 5.0),
 (0, 51, 5.0),
 (0, 364, 4.0),
 (0, 404, 4.0),
 (0, 641, 4.0),
 (0, 113, 5.0),
 (0, 7, 5.0),
 (0, 677, 3.0),
 (0, 331, 4.0)]

In [18]:
testset[:10]

[('551', '13', 1.0),
 ('830', '229', 2.0),
 ('868', '746', 2.0),
 ('130', '588', 4.0),
 ('759', '275', 4.0),
 ('373', '233', 3.0),
 ('351', '1105', 4.0),
 ('229', '269', 4.0),
 ('1', '53', 3.0),
 ('497', '25', 4.0)]

In [19]:
from surprise import SVD

algo = SVD()
algo.fit(trainset)

In [20]:
predictions = algo.test(testset) # == predict

In [23]:
type(predictions), type(predictions[0]), len(predictions)

(list, surprise.prediction_algorithms.predictions.Prediction, 20000)

In [24]:
predictions[:5]

[Prediction(uid='551', iid='13', r_ui=1.0, est=3.1720758487231224, details={'was_impossible': False}),
 Prediction(uid='830', iid='229', r_ui=2.0, est=3.31623401869121, details={'was_impossible': False}),
 Prediction(uid='868', iid='746', r_ui=2.0, est=3.0870150467154533, details={'was_impossible': False}),
 Prediction(uid='130', iid='588', r_ui=4.0, est=4.669945218768222, details={'was_impossible': False}),
 Prediction(uid='759', iid='275', r_ui=4.0, est=4.392773876462733, details={'was_impossible': False})]

In [25]:
[ (prediction.uid, prediction.iid, prediction.est) for prediction in predictions[:3] ]

[('551', '13', 3.1720758487231224),
 ('830', '229', 3.31623401869121),
 ('868', '746', 3.0870150467154533)]

In [26]:
algo.predict(str(759), str(275))

Prediction(uid='759', iid='275', r_ui=None, est=4.392773876462733, details={'was_impossible': False})

In [27]:
from surprise import accuracy

print( accuracy.rmse(predictions) )
print( accuracy.mae(predictions) )

RMSE: 0.9346
0.934585457159151
MAE:  0.7360
0.7359758052325709


In [28]:
import pandas as pd

ratings = pd.read_csv('data-files/ml-latest-small/ratings.csv')

In [29]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [31]:
ratings['rating'].describe()

count    100836.000000
mean          3.501557
std           1.042529
min           0.500000
25%           3.000000
50%           3.500000
75%           4.000000
max           5.000000
Name: rating, dtype: float64

In [30]:
# 원본 데이터에서 header 제거하고 파일로 저장
ratings.to_csv('data-files/ml-latest-small/ratings-noh.csv', header=False, index=False)

In [32]:
# 데이터 읽기 도구 + 읽기 설정 (파서) : Reader
from surprise import Reader

reader = Reader(line_format="user item rating timestamp", sep=",", rating_scale=(0.5, 5))
data = Dataset.load_from_file('data-files/ml-latest-small/ratings-noh.csv', reader)

In [36]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

alog = SVD(n_factors=50, random_state=42)

alog.fit(trainset)
predictions = alog.test(testset)

accuracy.rmse(predictions)
accuracy.mae(predictions)

RMSE: 0.8785
MAE:  0.6754


0.6753682865388253

In [37]:
movies = pd.read_csv('data-files/ml-latest-small/movies.csv')

In [38]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [46]:
rated_movies = ratings[ratings['userId'] == 1]['movieId'].tolist() # 1번 사용자가 평가한 영화 목록
total_movies = movies['movieId'].tolist() # 전체 영화 목록

not_rated_movies = [ movie for movie in total_movies if movie not in rated_movies ] # 1번 사용자가 평가하지 않은 영화 목록

In [47]:
len(rated_movies), len(not_rated_movies), len(total_movies)

(232, 9510, 9742)

In [48]:
predictions = [ algo.predict(str(1), str(movie)) for movie in not_rated_movies ] # 평가하지 않은 영화에 대해 예측

In [49]:
predictions[:5]

[Prediction(uid='1', iid='2', r_ui=None, est=3.3280536154906595, details={'was_impossible': False}),
 Prediction(uid='1', iid='4', r_ui=None, est=3.391013375959061, details={'was_impossible': False}),
 Prediction(uid='1', iid='5', r_ui=None, est=3.1386622727373976, details={'was_impossible': False}),
 Prediction(uid='1', iid='7', r_ui=None, est=4.248438652589138, details={'was_impossible': False}),
 Prediction(uid='1', iid='8', r_ui=None, est=3.3526428615885853, details={'was_impossible': False})]

In [53]:
print( predictions[0].est )
predictions.sort(key=lambda p: p.est, reverse=True) # 예상 평점을 기준으로 내림차순 정렬

4.912919318860377


In [56]:
top_10_recommended_movies = predictions[:10] # 예상 평점이 가장 높은 10개의 영화
top_10_recommended_movies

[Prediction(uid='1', iid='172', r_ui=None, est=4.912919318860377, details={'was_impossible': False}),
 Prediction(uid='1', iid='168', r_ui=None, est=4.819567325406738, details={'was_impossible': False}),
 Prediction(uid='1', iid='484', r_ui=None, est=4.788495595708425, details={'was_impossible': False}),
 Prediction(uid='1', iid='89', r_ui=None, est=4.767196172848772, details={'was_impossible': False}),
 Prediction(uid='1', iid='12', r_ui=None, est=4.692252094764415, details={'was_impossible': False}),
 Prediction(uid='1', iid='173', r_ui=None, est=4.684800223507431, details={'was_impossible': False}),
 Prediction(uid='1', iid='174', r_ui=None, est=4.680697205447778, details={'was_impossible': False}),
 Prediction(uid='1', iid='100', r_ui=None, est=4.653160740314666, details={'was_impossible': False}),
 Prediction(uid='1', iid='493', r_ui=None, est=4.626846786716127, details={'was_impossible': False}),
 Prediction(uid='1', iid='87', r_ui=None, est=4.624645317854345, details={'was_impos

In [68]:
movie_ids = [m.iid for m in top_10_recommended_movies]
est_list = [m.est for m in top_10_recommended_movies]
movie_titles = [ movies[movies["movieId"] == int(id)] for id in movie_ids ]

In [71]:
movie_titles[0]['title'].values[0]

'Johnny Mnemonic (1995)'

In [73]:
[ (id, est, title['title'].values[0] )for id, est, title in zip(movie_ids, est_list, movie_titles) ]

[('172', 4.912919318860377, 'Johnny Mnemonic (1995)'),
 ('168', 4.819567325406738, 'First Knight (1995)'),
 ('484', 4.788495595708425, 'Lassie (1994)'),
 ('89', 4.767196172848772, 'Nick of Time (1995)'),
 ('12', 4.692252094764415, 'Dracula: Dead and Loving It (1995)'),
 ('173', 4.684800223507431, 'Judge Dredd (1995)'),
 ('174', 4.680697205447778, 'Jury Duty (1995)'),
 ('100', 4.653160740314666, 'City Hall (1996)'),
 ('493', 4.626846786716127, 'Menace II Society (1993)'),
 ('87', 4.624645317854345, 'Dunston Checks In (1996)')]